Temos o problema da equação de Poisson 1D:

\begin{equation*}
  \begin{cases}
    \displaystyle \frac{d^2u}{dx^2} = 2 \text{ para x } \epsilon (-1,1) \\\\
    \displaystyle u(x) = 2 \text{ para x = -1} \\\\
    \displaystyle u(x) = 2 \text{ para x = 1}
  \end{cases}
\end{equation*}

Considerando o caso mais geral de equação linear de segunda ordem, temos:

\begin{equation}
  a(x)u''(x) + b(x)u'(x) + c(x)u(x) = f(x)
\end{equation}

juntamente com 2 condições de contorno, por exemplo, Dirichlet:

$u(a)= \alpha$, $u(b) = \beta$

Assim transferindo e adapatando para a equação de Pennes simplificada divimos a equação da seguinte forma:

\begin{equation}
   1u''(x) = 2
\end{equation}

Desse modo temos que:

$u_a = 2$ -> condição de contorno de Dirichlet a esquerda

$u_b = 2$ -> condição de contorno de Dirichlet a direita

$f(x) = 2$ -> lado direito da equação, nesse caso constante

Temos que no caso geral a matriz A é montada da seguinte maneira:

\begin{equation*}
  A =
  \begin{bmatrix}
    (h^2c_1 -2a_1) & (a_1 +hb_1/2) &  &  &  &\\
    (a_2 -hb_2/2) & (h^2c_2 -2a_2) & (a_2 +hb_2/2) &  &  & \\
    &  (a_3 -hb_3/2) &  (h^2c_3 -2a_3) & (a_3 +hb_3/2) &  & \\
    &  &  ... & ... & ... &\\
    &  &  &  (a_{m-1} -hb_{m-1}/2) & (h^2c_{m-1} -2a_{m-1}) & (a_{m-1} +hb_{m-1}/2)\\
    &  &  &  &  (a_m -hb_2/2) & (h^2c_m -2a_m)\\
  \end{bmatrix}
\end{equation*}

Já a matriz F no caso geral é montada seguindo o seguinte padrão:

\begin{equation*}
  f =
  \begin{bmatrix}
   f(x_1) - u_a/h^2\\
   f(x_2) \\
   f(x_3) \\
   ...  \\
   f(x_{m-1})  \\
   f(x_m) - u_b/h^2 \\
  \end{bmatrix}
\end{equation*}

Tomando $h = 0.25$, montamos a matriz A do sistema $Au=F$, da seguinte forma:

\begin{equation*}
  A = \frac{1}{h^2}
  \begin{bmatrix}
   -2 & 1 &  &  &  & &\\
    1 &-2 & 1 &  &  & & \\
    &  1 & -2 & 1 &  & & \\
    &   & 1 & -2 & 1 & & \\
    &  &  & 1 & -2 & 1 & \\
    &  &  &  & 1 & -2 & 1\\
    &  &  &  &  & 1 & -2\\
  \end{bmatrix}
\end{equation*}



A matriz F vai seguir o seguinte formato:

\begin{equation*}
  f =
  \begin{bmatrix}
   -30\\
   2 \\
   2 \\
   2  \\
   2  \\
   2  \\
   -30 \\
  \end{bmatrix}
\end{equation*}

Nesse sentido vamos seguir o seguinte algoritmo para resolução do problema:



1.   Montar o sistema $Au = F$
2.   Resolver esse sistema
3.   Aplicar condição de contorno em u

Para isso aplicamos o método de decomposição LU sem pivoteamento

In [1]:
%%writefile lu.h
#ifndef LU_H
#define LU_H

int detInd(int i, int j, int n);
void montaA(float *A, int n, float h);
void montaF(float *F, int n, float h, int ua, int ub, int fx);
void resolucao(float *A, float *F, float *u, int n, int ua, int ub);

#endif //LU_H

Writing lu.h


In [41]:
%%writefile lu.cpp
#include <iostream>
using namespace std;
#include <fstream>
#include <vector>

int detInd(int i, int j, int n) {
  return (i < 100 && j < 100) ? i*n + j : -1;
}

void montaA(float *A, int n, float h) {

  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      if(j>=i-1 && j<i+2) {
          A[detInd(i,j,n)] = i==j ?  (1/(h*h))*-2 : (1/(h*h))*1;
      }
      else A[detInd(i,j,n)] = 0;
    }
  }

  cout << "Matriz A montada: " << endl;
  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      cout << A[detInd(i,j,n)] << " ";
    }
    cout << endl;
  }
}

void montaF(float *F, int n, float h, int ua, int ub, int fx) {

  for(int i=1; i<n; i++) {
    F[i] = fx;
  }

  F[0] = fx - ua/(h*h);
  F[n-1] = fx - ub/(h*h);

  cout << endl;
  cout << "Matriz F montada: " << endl;
  for(int i=0; i<n; i++) {
    cout << F[i] << " ";
  }
}

void resolucao(float *A, float *F, float *u, int n, int ua, int ub) {
  float *L = new float[n*n];
  float *U = new float[n*n];

  cout << endl;
  cout << endl;
  cout << endl;

  //MONTAR AS MATRIZES L E U
  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      U[detInd(i,j,n)] = 0;
      L[detInd(i,j,n)] = 0;
    }
  }

  for(int i=0; i<n; i++) {
    L[detInd(i,i,n)] = 1;

    //U certinho
    for(int j=0; j<i+1; j++) {
      float s1 = 0;
      for(int k=0; k<j; k++) {
        s1 += U[detInd(k,i,n)] * L[detInd(j,k,n)];
      }
      U[detInd(j,i,n)] = A[detInd(j,i,n)] - s1;
    }

    for(int j=i; j<n; j++) {
      float s2 = 0;
      for(int k=0; k<i; k++) {
        s2 += U[detInd(k,i,n)] * L[detInd(j,k,n)];
      }
      L[detInd(j,i,n)] = (A[detInd(j,i,n)] - s2) / U[detInd(i,i,n)];
    }
  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz U: " << endl;

  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      cout << U[detInd(i,j,n)] << " ";
    }
    cout << endl;
  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz L: " << endl;

  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      cout << L[detInd(i,j,n)] << " ";
    }
    cout << endl;
  }

  //Por substituicao resolvemos Lx = F
  float *x = new float[n];

  for(int i=0; i<n; i++) {
    x[i] = 0;
  }

  for(int i=0; i<n; i++) {
    float s2 = 0;
    for(int j=0; j<i; j++) {
        s2 += L[detInd(i,j,n)]*x[j];
    }
    x[i] = (F[i] - s2)/L[detInd(i,i,n)];
  }

  cout << endl;
  cout << endl;
  cout << "Matriz y:" << endl;
  cout << endl;

  for(int i=0; i<n; i++) {
      cout << x[i] << " ";
  }

  cout << endl;
  cout << endl;
  cout << endl;

  //e assim por ultimo fazemos a retrosubstituicao Uu = x , Ux = b
  float *sol = new float[n];

  for(int i=0; i<n; i++) {
    sol[i] = 0;
  }

  for(int i=n-1; i>=0; i--) {
    float s3 = 0;
    for(int j=i+1; j<n; j++) {
      s3 += U[detInd(i,j,n)]*sol[j];
    }
    sol[i] = (x[i] - s3)/U[detInd(i,i,n)];
  }

  //sol[n+1] = ub;
  //sol[0] = ua;

  cout << endl;
  cout << endl;
  cout << "Matriz u:" << endl;
  cout << endl;

  for(int i=0; i<n; i++) {
      cout << sol[i] << " ";
  }

  delete [] L;
  delete [] U;
  delete [] x;
  delete [] sol;
}

Overwriting lu.cpp


In [82]:
%%writefile pivoteamento.h
#ifndef PIVOTEAMENTO_H
#define PIVOTEAMENTO_H

void substituicaoSucess(float *A, float *F, int *p, int n, float *x);
void substituicaoRetroa(float *A, float *F, float *x, int *p, int n);
void montaPLU(float *A, float *F, float *u, int n, int *p);

#endif //PIVOTEAMENTO_H

Overwriting pivoteamento.h


In [105]:
%%writefile pivoteamento.cpp
#include <iostream>
using namespace std;
#include <fstream>
#include <vector>
#include <math.h>

#include "lu.h"

//SUBSTITUICAO SUCESSIVA Lx = pF
void substituicaoSucess(float *A, float *F, int *p, int n, float *x) {
  for(int k=0; k<n; k++) {
    x[k] = F[p[k]];

    for(int j=0; j<k; j++) {
      x[k] = x[k] - A[detInd(p[k],j,n)]*x[j];
    }
    x[k] = x[k]/1.0; //pode retirar
  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz y: " << endl;
  for(int i=0; i<n; i++) {
    cout << x[i] << " ";
  }
}

//SUBSTITUICAO RETROATIVA Ux = F
void substituicaoRetroa(float *A, float *F, float *x, int *p, int n) {
  for(int k=n-1; k>=0; k--) {

    for(int j=k+1; j<n; j++) {
      x[k] = x[k] -A[detInd(p[k],j,n)]*x[j];
    }
    x[k] = x[k]/A[detInd(p[k],k,n)];

  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz solucao: " << endl;
  for(int i=0; i<n; i++) {
    cout << x[i] << " ";
  }
}


void montaPLU(float *A, float *F, float *u, int n, int *p) {
  for(int k=0; k<n; k++) {
    //ENCONTRAR PIVOS DE A
    float max = fabs(A[detInd(p[k],k,n)]);
    int iMax = k;

    for(int i=k+1; i<n; i++) {
      if(max<fabs(A[detInd(p[i],k,n)])) {
        max = fabs(A[detInd(p[i],k,n)]);
        iMax = i;
      }
      //cout << endl;
      //cout << endl;
      //cout << endl;
      //cout << "Maior pivo: " << max << endl;
    }
    //cout << "Indice: " << iMax << endl;

    int tmp = p[k];
    p[k] = p[iMax];
    p[iMax] = tmp;

    //anular elementos
    for(int i=k+1; i<n; i++) {
      float m = A[detInd(p[i],k,n)]/A[detInd(p[k],k,n)];
      //F[i] = F[p[k]] - F[p[i]];
      A[detInd(p[i],k,n)] = m;
      for(int j=k+1; j<n; j++) {
        A[detInd(p[i],j,n)] = A[detInd(p[i],j,n)] - (m*A[detInd(p[k],j,n)]);
      }

    }
  }

  cout << endl;
  cout << endl;
  cout << endl;
  cout << "Matriz A(LU): " << endl;
  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      cout << A[detInd(p[i],j,n)] << ", ";
    }
    cout << endl;
  }
}

Overwriting pivoteamento.cpp


In [112]:
%%writefile main.cpp
#include <iostream>
using namespace std;
#include <string>
#include <cmath>

#include "lu.h"
#include "pivoteamento.h"

int main()
{

    int n = 7; //tamanho do dominio
    float h = 0.25;
    int ua = 2; //condicao de contorno dirichelet a esquerda
    int ub = 2; //condicao de contorno dirichelet a direita
    int fx = 2; //lado direito da equacao que eh constante

    float *A = new float[n*n];
    float *F = new float[n];
    float *u = new float[n+2];
    int *p = new int[n];

    for(int i=0; i<n; i++) {
      p[i] = i;
    }

    //A[detInd(0,0,n)] = 1;
    //A[detInd(0,1,n)] = -2;
    //A[detInd(0,2,n)] = 7;
    //A[detInd(0,3,n)] = 2;
    //A[detInd(1,0,n)] = 2;
    //A[detInd(1,1,n)] = 5;
    //A[detInd(1,2,n)] = -3;
    //A[detInd(1,3,n)] = 1;
    //A[detInd(2,0,n)] = 9;
    //A[detInd(2,1,n)] = -6;
    //A[detInd(2,2,n)] = 4;
    //A[detInd(2,3,n)] = 1;
    //A[detInd(3,0,n)] = 4;
    //A[detInd(3,1,n)] = -3;
    //A[detInd(3,2,n)] = -6;
    //A[detInd(3,3,n)] = 7;

    //F[0] = -18;
    //F[1] = 31;
    //F[2] = 35;
    //F[3] = 15;

    float *x = new float[n];

    for(int i=0; i<n; i++) {
      x[i] = 0;
    }

    montaA(A, n, h);
    montaF(F, n, h, ua, ub, fx);
    //resolucao(A, F, u, n, ua, ub);
    montaPLU(A, F, u, n, p);
    substituicaoSucess(A, F, p, n, x);
    substituicaoRetroa(A, F, x, p, n);


    //aplica as condicoes de contorno
    u[0] = ua;
    u[n+1] = ub;

    delete [] A;
    delete [] F;
    delete [] u;
    delete [] p;

    return 0;
}

Overwriting main.cpp


In [113]:
!g++ lu.cpp \
  pivoteamento.cpp \
  main.cpp \
  -o decompLu

In [114]:
!./decompLu

Matriz A montada: 
-32 16 0 0 0 0 0 
16 -32 16 0 0 0 0 
0 16 -32 16 0 0 0 
0 0 16 -32 16 0 0 
0 0 0 16 -32 16 0 
0 0 0 0 16 -32 16 
0 0 0 0 0 16 -32 

Matriz F montada: 
-30 2 2 2 2 2 -30 


Matriz A(LU): 
-32, 16, 0, 0, 0, 0, 0, 
-0.5, -24, 16, 0, 0, 0, 0, 
-0, -0.666667, -21.3333, 16, 0, 0, 0, 
-0, -0, -0.75, -20, 16, 0, 0, 
-0, -0, -0, -0.8, -19.2, 16, 0, 
-0, -0, -0, -0, -0.833333, -18.6667, 16, 
-0, -0, -0, -0, -0, -0.857143, -18.2857, 



Matriz y: 
-30 -13 -6.66667 -3 -0.4 1.66667 -28.5714 


Matriz solucao: 
1.5625 1.25 1.0625 1 1.0625 1.25 1.5625 